In [76]:
import numba
from time import time

numba.__version__

'0.42.0'

In [2]:
from numba import jitclass, njit         # import the decorator
from numba import int32, float32
import numpy as np

In [73]:
import functools

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

In [107]:
spec = [
    ('work', int32),          # an array field
    ('age', int32)
]

@jitclass(spec)
class person(object):
    def __init__(self):
        self.age = 10
        self.work = 0
        
    def aged(self):
        self.age += 1
    
    def worked(self):
        if np.random.random() > 0.5:
            self.work = True
        else:
            self.work = 0

tmplist = [person(),person()]
type_list = numba.typeof(tmplist)
print(type_list)
spec2 = [
    ('pop', type_list)
]
@jitclass(spec2)
class arraybag(object):
    def __init__(self,value):
        self.pop = [person() for i in range(value)]
    
    def add_person(self,person):    
        self.pop.append(person)
        
    def print_person(self):
        for pers in self.pop:
            print(pers.age)
    def len(self):
        return len(self.pop)
    def aged(self):
        for pers in self.pop:
            pers.aged()
    def worked(self):
        for pers in self.pop:
            pers.worked()
    def run_transtion(self,method):
        for pers in self.pop:
            getattr(pers,method)()
    def run(self,year):
        for i in range(year):
            self.aged()
            self.worked()
    def to_python(self):
        pop_out = [pers for pers in self.pop]        
        return pop_out
    def to_python2(self):
        return self.pop
type_arraybag = numba.typeof(arraybag(1))
@njit
def to_python(population):
    return population
@njit
def aged(pop):
        for pers in pop.pop:
            pers.aged()

@njit
def worked(pop):
    for pers in pop.pop:
        pers.worked()
@njit
def run(pop,year):
    for i in range(year):
        aged(pop)
        worked(pop)

reflected list(instance.jitclass.person#7f8e38a0b178<work:int32,age:int32>)


In [15]:
one = person()

In [47]:
numba.typeof(listperson)

reflected list(instance.jitclass.person#7f8e2d1a0c88<work:int32,age:int32>)

In [40]:
numba.typeof(all)

instance.jitclass.arraybag#7f8e32aadc28<pop:reflected list(instance.jitclass.person#7f8e32a9d228<work:int32,age:int32>),value:int32>

In [44]:
#listperson = [person() for i in range(1000000)]


elapse time: 0.8529849052429199


In [108]:
start = time()
all = arraybag(10)
end =time()
print("elapse time: "+str(end-start))


elapse time: 6.198883056640625e-05


In [95]:
start = time()
all.run(20)
end =time()
print("elapse time: "+str(end-start))

elapse time: 1.155623197555542


In [96]:
start = time()
run(all,20)
end =time()
print("elapse time: "+str(end-start))

elapse time: 1.1042931079864502


In [109]:
all.run_transtion("aged")

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'getattr':[0m [1m[1mcannot determine Numba type of <class 'builtin_function_or_method'>[0m
[1m
File "<ipython-input-107-4745d47054e5>", line 48:[0m
[1m    def run_transtion(self,method):
        <source elided>
        for pers in self.pop:
[1m            getattr(pers,method)()
[0m            [1m^[0m[0m
[0m
[0m[1m[1] During: resolving callee type: BoundFunction((<class 'numba.types.misc.ClassInstanceType'>, 'run_transtion') for instance.jitclass.arraybag#7f8e3903c2a8<pop:reflected list(instance.jitclass.person#7f8e38a0b178<work:int32,age:int32>)>)[0m
[0m[1m[2] During: typing of call at <string> (3)
[0m
[1m
File "<string>", line 3:[0m
[1m<source missing, REPL/exec in use?>[0m

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolv

In [103]:
all.print_person()

10
10
10
10
10
10
10
10
10
10


In [99]:
numba.typeof(all.pop)

reflected list(instance.jitclass.person#7f8e353dd0c8<work:int32,age:int32>)

In [69]:
start = time()
all.run(20)
end =time()
print("elapse time: "+str(end-start))

elapse time: 7.009506225585938e-05


In [70]:
start = time()
all.run(20)
end =time()
print("elapse time: "+str(end-start))

In [105]:
pop2 = all.to_python2()
pop2

In [59]:
pop

In [49]:
all.pop[1].age

10

In [32]:
run(all,2)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1mInvalid use of getiter with parameters (BoundFunction(list.pop for reflected list(instance.jitclass.person#7f8e32a9d228<work:int32,age:int32>)))
 * parameterized[0m
[0m[1m[1] During: typing of intrinsic-call at <ipython-input-30-58c7ccac181e> (61)[0m
[1m
File "<ipython-input-30-58c7ccac181e>", line 61:[0m
[1mdef aged(pop):
[1m        for pers in pop.pop:
[0m        [1m^[0m[0m

[0m[1m[1] During: resolving callee type: type(CPUDispatcher(<function aged at 0x6084a6d08>))[0m
[0m[1m[2] During: typing of call at <ipython-input-30-58c7ccac181e> (71)
[0m
[1m
File "<ipython-input-30-58c7ccac181e>", line 71:[0m
[1mdef run(pop,year):
    <source elided>
    for i in range(year):
[1m        aged(pop.pop)
[0m        [1m^[0m[0m

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported fea